https://deep-learning-study.tistory.com/686

https://stackoverflow.com/questions/71818652/attributeerror-module-torchtext-has-no-attribute-legacy
torchtext import 안될때 버전변경

https://codlingual.tistory.com/91

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k#stack-overflow 형님들은 신임
from torchtext.legacy.data import Field, BucketIterator
import spacy
import numpy as np
import random
import math
import time

In [2]:
seed=1234

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)#pytorch의 randomseed고정
torch.cuda.manual_seed(seed)#pytorch randomseed 고정
torch.backends.cudnn.deterministic=True#cudnn randomseed 를 고정

python -m spacy download en

python -m spacy download de

In [7]:
import de_core_news_sm
import en_core_web_sm

spacy_en= en_core_web_sm.load()
spacy_de=de_core_news_sm.load()

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]#reverse

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

SRC=Field(tokenize=tokenize_de, init_token='<sos>', eos_token='<sos>', lower=True)
TRG=Field(tokenize=tokenize_en, init_token='<sos>', eos_token='<sos>', lower=True)

In [8]:
train_data, valid_data, test_data=Multi30k.splits(exts=('.de','.en'),fields=(SRC,TRG))
# train, validation, test 데이터를 불러오고, 다운로드 합니다.
# Multi30k dataset을 사용하여, 30,000개의 영어, 독일, 프랑스어 문장을 포함합니다.

downloading training.tar.gz


C:\Users\Lee_JongWoo\private\20221107\.data\multi30k\training.tar.gz: 100%|████████| 1.21M/1.21M [00:07<00:00, 165kB/s]


downloading validation.tar.gz


C:\Users\Lee_JongWoo\private\20221107\.data\multi30k\validation.tar.gz: 100%|█████| 46.3k/46.3k [00:00<00:00, 80.3kB/s]


downloading mmt_task1_test2016.tar.gz


C:\Users\Lee_JongWoo\private\20221107\.data\multi30k\mmt_task1_test2016.tar.gz: 100%|█| 66.2k/66.2k [00:01<00:00, 53.7k


In [23]:
print(vars(test_data.examples[0]))#입력 src는 입력단어가 거꾸로

{'src': ['.', 'anstarrt', 'etwas', 'der', ',', 'hut', 'orangefarbenen', 'einem', 'mit', 'mann', 'ein'], 'trg': ['a', 'man', 'in', 'an', 'orange', 'hat', 'starring', 'at', 'something', '.']}


In [15]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


In [9]:
print(f'Number of training examples:{len(train_data.examples)}')
print(f'Number of validation examples:{len(valid_data.examples)}')
print(f'Number of testing examples:{len(test_data.examples)}')

Number of training examples:29000
Number of validation examples:1014
Number of testing examples:1000


In [10]:
SRC.build_vocab(train_data, min_freq=2)
TRG.build_vocab(train_data,min_freq=2)
#토큰화 전처리가 끝나면, 각단어에 고유한 정수를 맵핑해주는 정수 인코딩작업이 필요하다. 그리고 이 전처리를 위해서는 단어집합이 필요한데, 
#정의한 filed에 build_vocab을 이용하면 단어집합을 만들수 있다.
# min_freq=2는 2번 이상 등장한 토큰을 출력합니다.
# 토큰이 1번만 등장했다면 <unk>로 대체합니다.

In [11]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

batch_size=128
train_iterator, valid_iterator, test_iterator=BucketIterator.splits((train_data, valid_data, test_data),batch_size=batch_size,device=device)

In [12]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim=hid_dim
        self.n_layers=n_layers
        #임베딩:입력값을 emb_dim벡터로 변경
        self.embedding=nn.Embeding(input_dim, emb_dim)
        
        #embeding을 입력받아 hid_dim크기의 hidden state,cell출력
        self.rnn=nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        
        self.dropout=nn.Dropout(dropout)
    
    def forward(self,src):
        #src=[src.len,batch_size]
        embedded= self.dropout(self.embedding(src))
        #initial hidden state는 제로텐서
        outputs, (hidden,cell)=self.rnn(embedded)
        # output: [src_len, batch_size, hid dim * n directions]
        # hidden: [n layers * n directions, batch_size, hid dim]
        # cell: [n layers * n directions, batch_size, hid dim]
        return hidden, cell

In [13]:
class Decoder(nn.Module):
    def __init__(self, out_dim, emb_dim, hid_dim, n_layers, dropouts):
        super().__init__()
        
        self.output_dim=output_dim
        self.hid_dim=hid_dim
        self.n_layers=n_layers
        
        #context vector를 받아  embdim으로 출력
        self.embedding=nn.Embeding(output_dim,emb_dim)
        self.rnn=nn.LSTM(emb_dim, hid_dim, n_layers, dropout=dropout)
        self.fc_out=nn.Linear(hid_dim, output_dim)
        
        self.dropout=nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        input= input.unsqueeze(0) #입력 [1,batchsize] 첫번째 input 은 <sos>
        
        embedded=self.dropout(self.embedding(input))
        
        output, (hidden,cell)= self.rnn(embedded, (hidden,cell))
        
        prediction=self.fc_out(output.squeeze(0))
        
        return prediction, hidden, cell

In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super.__init__()
        
        self.encoder=encoder
        self.decoder=decoder
        self.device=device
         # encoder와 decoder의 hid_dim이 일치하지 않는 경우 에러메세지
        assert encoder.hidden_dim == decoder.hidden_dim, \
            'Hidden dimensions of encoder decoder must be equal'
         # encoder와 decoder의 hid_dim이 일치하지 않는 경우 에러메세지
        assert encoder.n_layers == decoder.n_layers, \
            'Encoder and decoder must have equal number of layers'
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        #target word(Ground Truth)를 디코더의 다음 입력으로 넣어주는 기법
        #src=[src len,batch_size]
        #trg=[trg len,batch_size]
        
        batch_size=trg.shape[1]
        trg_len=trg.shape[0] # 타겟 토큰 길이 얻기
        trg_vocab_size=self.decoder.output_dim # context vector의 차원
        # decoder의 output을 저장하기 위한 tensor
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        # initial hidden state
        hidden, cell=self.encoder(src)
        
        input=trg[0,:]
        
        for t in range(1, trg_len):# <eos> 제외하고 trg_len-1 만큼 반복
            output, hidden, cell = self.encoder(input, hidden, cell)
            
            outputs[t]=output# prediction 저장
            teacher_force=random.randorm()<teacher_forcing_ratio# teacher forcing을 사용할지, 말지 결정
            top1=output.argmax[1]# 가장 높은 확률을 갖은 값 얻기
            input=trg[t] if teacher_force else top1# teacher forcing의 경우에 다음 lstm에 target token 입력
            
        return outputs